# Projeto de Trainee - Processamento de Linguagem Natural
Por: Camilla Fonseca


<img src="https://i.ibb.co/DtHQ3FG/802x265-Logo-GT.png" width="370">


### Ánalise de sentimentos com um dataset de reviews do IMDB.

# Parte 1
Pré-processamento, feature extraction e aplicação de modelos.

# Pré-processamento


In [ ]:
# Importando bibliotecas
import pandas as pd
import re
import nltk
from bs4 import BeautifulSoup
import spacy

In [ ]:
spc_en = spacy.load('en')

### Visão geral do dataset

In [ ]:
data = pd.read_csv("/content/drive/My Drive/nlp-trainee/IMDB Dataset.csv")

In [ ]:
data.head(15)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


Nenhuma entrada nula no dataset.

In [ ]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


A distibuição de *sentiment* está surpreendentemente equilibrada: exatamente 50% dos dados para cada categoria (*positive* e *negative*).

Me chamou a atenção haver reviews que se repetem, me parece estranho ter reviews *exatamente* iguais, por isso tratei como duplicatas e removi.

In [ ]:
duplicatas = data.duplicated(subset = 'review', keep = False)

data[duplicatas].sort_values(by = 'review')

,review,sentiment
34058,"""Go Fish"" garnered Rose Troche rightly or wron...",negative
47467,"""Go Fish"" garnered Rose Troche rightly or wron...",negative
29956,"""Three"" is a seriously dumb shipwreck movie. M...",negative
31488,"""Three"" is a seriously dumb shipwreck movie. M...",negative
47527,"""Witchery"" might just be the most incoherent a...",negative
...,...,...
47876,this movie sucks. did anyone notice that the e...,negative
44122,"well, the writing was very sloppy, the directi...",negative
23056,"well, the writing was very sloppy, the directi...",negative
10163,"when I first heard about this movie, I noticed...",positive


In [ ]:
# Retirando duplicados completamente iguais
data.drop_duplicates(inplace=True)

duplicatas = data.duplicated(subset = 'review', keep = False)
data[duplicatas].sort_values(by = 'review')

,review,sentiment


As duplicatas foram todas removidas. Checando novamente informações sobre as features:

In [ ]:
data.describe()

,review,sentiment
count,49582,49582
unique,49582,2
top,"How do I describe the horrors?!!! First, some ...",positive
freq,1,24884


### Remover tags html com BeautifulSoup

In [ ]:
def remove_tags_html(texto):
    """(str) -> str
    Recebe uma string e retira tags html se houver."""
    soup = BeautifulSoup(texto, "html.parser")
    sem_tags = soup.get_text(separator=" ")
    return sem_tags

### Selecionar apenas letras do alfabeto em lowercase

In [ ]:
def apenas_letras_lowercase(texto):
  '''(str) -> list
  Recebe texto, converte tudo para letras minusculas, seleciona apenas
  letras do alfabeto e retorna os tokens.
  Como o dataset esta em ingles nao e preciso se preocupar com acentos.'''
  return re.findall(r'[a-z]+', texto.lower())

### Remover stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.corpus.stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Achei melhor conferir a lista de stopwords, porque palavras como "no", "not" e verbos na negativa como "don't" são importantes para a análise de sentimentos. Sem elas, uma review negativa como "I don't like this film" acabaria sendo classificada como positiva. Minha abordagem então vai ser expandir contrações negativas (algo a ser feito antes de selecionar apenas letras) e tirar as negações das stopwords.

In [ ]:
def descontrai_negativas(texto):
    """(str) -> str
    Expande palavras contraidas, ex: don't -> do not."""
    return re.sub(r"n\'t", " not", texto)

In [ ]:
def remove_stopwords(tokens):
  '''(list) -> str
  Recebe lista de tokens de um texto e devolve uma string com o texto
  sem stopwords.'''
  
  stopwords = nltk.corpus.stopwords.words('english')
  # tirar 'no' e 'not' da lista
  stopwords.remove('no')
  stopwords.remove('not')

  stop = set(stopwords)

  meaningful_words = [w for w in tokens if w not in stop]
  return " ".join(meaningful_words)


### Lematização

In [ ]:
def lematizacao(texto):
  '''(str) -> str
  Recebe um texto e retorna o texto após o processo de lematizacao'''

  # Instanciando o objeto spacy
  spc_texto = spc_en(texto)

  # Lematizando
  tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto]

  # Juntando os tokens e retornando
  return " ".join(tokens)

### Função que faz todos os pré-processamentos

In [ ]:
def pre_processamento(texto):
  '''(str) -> str
  Recebe uma string e retorna essa string pre-processada.'''

  # Remover as tags
  texto = remove_tags_html(texto)

  # Expandir as contracoes de negacao
  texto = descontrai_negativas(texto)

  # Remover caracteres que nao sao letras e tokenizacao
  tokens = apenas_letras_lowercase(texto)

  # Remover stopwords
  texto = remove_stopwords(tokens)

  # Lematizacao
  texto = lematizacao(texto)

  return texto

### Aplicando na coluna review

In [ ]:
data['review'] = data['review'].apply(pre_processamento)

In [ ]:
data.head(15)

,review,sentiment
0,one reviewer mention watch oz episode hook rig...,positive
1,wonderful little production filming technique ...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically family little boy jake think zombie ...,negative
4,petter mattei love time money visually stunnin...,positive
5,probably time favorite movie story selflessnes...,positive
6,sure would like see resurrection date seahunt ...,positive
7,show amazing fresh innovative idea first air f...,negative
8,encouraged positive comment film look forward ...,negative
9,like original gut wrenching laughter like movi...,positive


In [ ]:
# vamo salvar pq ngm merece ficar rodando isso sempre
data.to_csv('imdb_preprocessado.csv')
!cp imdb_preprocessado.csv "/content/drive/My Drive/nlp-trainee"

In [ ]:
data = pd.read_csv("/content/drive/My Drive/nlp-trainee/imdb_preprocessado.csv")
data.drop('Unnamed: 0',1, inplace=True)

# Feature extraction


Podemos fazer usando Bag of Words e um vetor de componentes binários ou TF-IDF.

### Com one-hot encoding

In [ ]:
# Importando o CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Instanciando o CountVectorizer, binary=True faz a codificacao one-hot
vectorizer = CountVectorizer(binary=True, max_features=5000)

texto = data['review']

# Vetorizando o texto
X_onehot = vectorizer.fit_transform(texto)

In [ ]:
X_onehot.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
print(X_onehot.shape, type(X_onehot))

(49582, 5000) <class 'scipy.sparse.csr.csr_matrix'>


### Com TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Instanciando o TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=5000)

# Vetorizando
X_tfidf = tfidf_vect.fit_transform(texto)

In [ ]:
print(X_tfidf)

  (0, 4029)	0.054994928920294124
  (0, 4557)	0.05819529381464423
  (0, 4478)	0.07506396225427871
  (0, 4779)	0.08482148011274493
  (0, 4658)	0.08654915520260668
  (0, 864)	0.08833831540668428
  (0, 391)	0.04305156038112942
  (0, 2784)	0.03694235107264837
  (0, 1595)	0.05445524386718797
  (0, 4075)	0.07145695038230758
  (0, 4277)	0.06287588428286174
  (0, 2529)	0.05231232888772138
  (0, 451)	0.09434157759354297
  (0, 4630)	0.042382283853734194
  (0, 793)	0.06125982437066916
  (0, 2837)	0.06022512880509209
  (0, 4874)	0.027277436611936937
  (0, 3142)	0.05781203192180805
  (0, 2497)	0.04609525563460456
  (0, 2309)	0.19839413847677118
  (0, 3933)	0.06779781442704931
  (0, 1995)	0.07889433579858762
  (0, 1963)	0.07313272617886019
  (0, 2601)	0.05754464563714906
  (0, 1902)	0.0770379823243753
  :	:
  (49581, 612)	0.08807406790394287
  (49581, 4222)	0.08768553840596277
  (49581, 339)	0.06251163182145193
  (49581, 2575)	0.085723462250591
  (49581, 1939)	0.09936555833177468
  (49581, 2514)	0.06

# Aplicando modelos

In [ ]:
# tranformando positive em 1 e negative em 0
data['sentiment'] = pd.get_dummies(data['sentiment'])['positive']

Vou testar os modelos tanto para os dados vetorizados com one-hot como para os com tf-idf. Mas primeiro, é preciso dividir os dados em base de treino (70%) e teste (30%).

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_onehot, data['sentiment'],
                                                        test_size=0.3, random_state = 10)
# Vou usar a mesma seed para comparar melhor
X2_train, X2_test, y2_train, y2_test = train_test_split(X_tfidf, data['sentiment'],
                                                        test_size=0.3, random_state = 10)

In [ ]:
# Importando as métricas a serem usadas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Regressão Logística

### Texto vetorizado com one-hot

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Instanciando a reg. logistica
reglog = LogisticRegression()

# Aplicando o modelo
reglog.fit(X1_train, y1_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# Predicao
y1_reglog_pred = reglog.predict(X1_test)

Vamos agora analisar as métricas:

In [ ]:
def mostra_metricas(y_true, y_pred):
  print("Matriz de confusão:")
  print(confusion_matrix(y_true, y_pred))
  print("\nAcurácia: ", accuracy_score(y_true, y_pred))
  print("\n",classification_report(y_true, y_pred))

In [ ]:
mostra_metricas(y1_test, y1_reglog_pred)

Matriz de confusão:
[[6422 1042]
 [ 929 6482]]

Acurácia:  0.8674957983193278

               precision    recall  f1-score   support

           0       0.87      0.86      0.87      7464
           1       0.86      0.87      0.87      7411

    accuracy                           0.87     14875
   macro avg       0.87      0.87      0.87     14875
weighted avg       0.87      0.87      0.87     14875



### Texto vetorizado com tf-idf

In [ ]:
reglog2 = LogisticRegression()

reglog2.fit(X2_train, y2_train)

y2_reglog_pred = reglog2.predict(X2_test)

In [ ]:
mostra_metricas(y2_test, y2_reglog_pred)

Matriz de confusão:
[[6503  961]
 [ 746 6665]]

Acurácia:  0.8852436974789916

               precision    recall  f1-score   support

           0       0.90      0.87      0.88      7464
           1       0.87      0.90      0.89      7411

    accuracy                           0.89     14875
   macro avg       0.89      0.89      0.89     14875
weighted avg       0.89      0.89      0.89     14875



A diferença não é gritante, mas o desempenho do modelo foi um pouco melhor com tf-idf: tanto a acurácia quanto o f1 são maiores, e a matriz de confusão indica menor número de falsos positivos e negativos.

## Naive Bayes Gaussiano

### One-hot

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb1 = GaussianNB()

gnb1.fit(X1_train.toarray(), y1_train)

y1_gnb_pred = gnb1.predict(X1_test.toarray())

In [ ]:
mostra_metricas(y1_test, y1_gnb_pred)

Matriz de confusão:
[[6205 1259]
 [1994 5417]]

Acurácia:  0.7813109243697479

               precision    recall  f1-score   support

           0       0.76      0.83      0.79      7464
           1       0.81      0.73      0.77      7411

    accuracy                           0.78     14875
   macro avg       0.78      0.78      0.78     14875
weighted avg       0.78      0.78      0.78     14875



### Tf-idf

In [ ]:
gnb2 = GaussianNB()

gnb2.fit(X2_train.toarray(), y2_train)

y2_gnb_pred = gnb2.predict(X2_test.toarray())

mostra_metricas(y2_test, y2_gnb_pred)

Matriz de confusão:
[[5937 1527]
 [1648 5763]]

Acurácia:  0.7865546218487395

               precision    recall  f1-score   support

           0       0.78      0.80      0.79      7464
           1       0.79      0.78      0.78      7411

    accuracy                           0.79     14875
   macro avg       0.79      0.79      0.79     14875
weighted avg       0.79      0.79      0.79     14875



A diferença aqui é menor ainda, mas parece que com tf-idf foi ligeiramente melhor. Quanto às predições incorretas, as matrizes de confusão indicam que com one-hot houve um número maior de falsos negativos, enquanto com tf-idf foi mais balanceado.

## Árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

### One-hot

In [ ]:
dt1 = DecisionTreeClassifier()

dt1.fit(X1_train, y1_train)

y1_dt_pred = dt1.predict(X1_test)

mostra_metricas(y1_test, y1_dt_pred)

Matriz de confusão:
[[5295 2169]
 [2105 5306]]

Acurácia:  0.7126722689075631

               precision    recall  f1-score   support

           0       0.72      0.71      0.71      7464
           1       0.71      0.72      0.71      7411

    accuracy                           0.71     14875
   macro avg       0.71      0.71      0.71     14875
weighted avg       0.71      0.71      0.71     14875



### Tf-idf

In [ ]:
dt2 = DecisionTreeClassifier()

dt2.fit(X2_train, y2_train)

y2_dt_pred = dt2.predict(X2_test)

mostra_metricas(y2_test, y2_dt_pred)

Matriz de confusão:
[[5293 2171]
 [2138 5273]]

Acurácia:  0.7103193277310924

               precision    recall  f1-score   support

           0       0.71      0.71      0.71      7464
           1       0.71      0.71      0.71      7411

    accuracy                           0.71     14875
   macro avg       0.71      0.71      0.71     14875
weighted avg       0.71      0.71      0.71     14875



A diferença é novamente quase nula, mas agora parece que com one-hot foi ligeiramente melhor.

## Resultados

O melhor modelo foi a regressão logística, seguido do naive bayes gaussiano e por último a árvore de decisão. Os melhores resultados foi obtido pela regressão logística com tf-idf: acurácia de 88,5%, f1 89% e matriz de confusão com menores números tanto de falsos positivos como negativos.